<a href="https://colab.research.google.com/github/VOTUANANH01/OLAP/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <h1> Import dataset

In [1]:
# link dataset: https://data.cityofnewyork.us/Transportation/2020-Green-Taxi-Trip-Data/pkmi-4kfn
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/MyDrive/OLAP"

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1wiNeQ2WA_pHrom0EY08CwBqOVB5hicqa/OLAP


In [2]:
!git clone https://github.com/VOTUANANH01/OLAP.git
!unzip "/content/OLAP/2020_Green_Taxi_Trip_Data.zip"

fatal: destination path 'OLAP' already exists and is not an empty directory.
Archive:  /content/OLAP/2020_Green_Taxi_Trip_Data.zip
  inflating: 2020_Green_Taxi_Trip_Data.csv  


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import samples

In [5]:
df=pd.read_csv("/content/2020_Green_Taxi_Trip_Data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,12/18/2019 03:52:30 PM,12/18/2019 03:54:39 PM,N,1.0,264,264,5.0,0.00,3.50,0.50,0.5,0.01,0.0,NaN,0.3,4.81,1.0,1.0,0.00
1,2.0,01/01/2020 12:45:58 AM,01/01/2020 12:56:39 AM,N,5.0,66,65,2.0,1.28,20.00,0.00,0.0,4.06,0.0,NaN,0.3,24.36,1.0,2.0,0.00
2,2.0,01/01/2020 12:41:38 AM,01/01/2020 12:52:49 AM,N,1.0,181,228,1.0,2.47,10.50,0.50,0.5,3.54,0.0,NaN,0.3,15.34,1.0,1.0,0.00
3,1.0,01/01/2020 12:52:46 AM,01/01/2020 01:14:21 AM,N,1.0,129,263,2.0,6.30,21.00,3.25,0.5,0.00,0.0,NaN,0.3,25.05,2.0,1.0,2.75
4,1.0,01/01/2020 12:19:57 AM,01/01/2020 12:30:56 AM,N,1.0,210,150,1.0,2.30,10.00,0.50,0.5,0.00,0.0,NaN,0.3,11.30,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734046,NaN,12/31/2020 11:02:00 PM,12/31/2020 11:16:00 PM,NaN,NaN,174,168,NaN,7.04,20.33,0.00,0.0,0.00,0.0,NaN,0.3,20.63,NaN,NaN,NaN
1734047,NaN,12/31/2020 11:39:00 PM,12/31/2020 11:54:00 PM,NaN,NaN,256,225,NaN,2.79,13.17,0.00,0.0,0.00,0.0,NaN,0.3,13.47,NaN,NaN,NaN
1734048,NaN,12/31/2020 11:09:00 PM,12/31/2020 11:33:00 PM,NaN,NaN,146,10,NaN,12.41,45.12,0.00,0.0,2.75,0.0,NaN,0.3,48.17,NaN,NaN,NaN
1734049,NaN,12/31/2020 11:01:00 PM,12/31/2020 11:12:00 PM,NaN,NaN,196,28,NaN,4.46,13.96,0.00,0.0,3.13,0.0,NaN,0.3,17.39,NaN,NaN,NaN


In [7]:
decode_df=df.copy()
decode_df=decode_df[["VendorID","store_and_fwd_flag","RatecodeID","payment_type","trip_type"]]
decode_df

,VendorID,store_and_fwd_flag,RatecodeID,payment_type,trip_type
0,2.0,N,1.0,1.0,1.0
1,2.0,N,5.0,1.0,2.0
2,2.0,N,1.0,1.0,1.0
3,1.0,N,1.0,2.0,1.0
4,1.0,N,1.0,1.0,1.0
...,...,...,...,...,...
1734046,NaN,NaN,NaN,NaN,NaN
1734047,NaN,NaN,NaN,NaN,NaN
1734048,NaN,NaN,NaN,NaN,NaN
1734049,NaN,NaN,NaN,NaN,NaN


In [8]:
decode_columns={ "VendorID": {1.0:"Creative Mobile Technologies, LLC;",2.0:"VeriFone Inc"},
"store_and_fwd_flag":{"Y":"store and forward trip","N":"not a store and forward trip"},
"payment_type": {1.0: "Credit card",
2.0: "Cash",
3.0: "No charge",
4.0: "Dispute",
5.0: "Unknown",
6.0: "Voided trip"},
"RatecodeID":{1.0: "Standard rate",
2.0:"JFK",
3.0:"Newark",
4.0:"Nassau or Westchester",
5.0:"Negotiated fare",
6.0:"Group ride"
},
"trip_type":{1.0: "Street-hail",
2.0: "Dispatch"}}

In [9]:
decode_df_new=decode_df.replace(decode_columns)
decode_df_new.columns = [x + "_desc" for x in decode_df_new.columns]
final_df=pd.concat([decode_df,decode_df_new],axis=1)
final_df

,VendorID,store_and_fwd_flag,RatecodeID,payment_type,trip_type,VendorID_desc,store_and_fwd_flag_desc,RatecodeID_desc,payment_type_desc,trip_type_desc
0,2.0,N,1.0,1.0,1.0,VeriFone Inc,not a store and forward trip,Standard rate,Credit card,Street-hail
1,2.0,N,5.0,1.0,2.0,VeriFone Inc,not a store and forward trip,Negotiated fare,Credit card,Dispatch
2,2.0,N,1.0,1.0,1.0,VeriFone Inc,not a store and forward trip,Standard rate,Credit card,Street-hail
3,1.0,N,1.0,2.0,1.0,"Creative Mobile Technologies, LLC;",not a store and forward trip,Standard rate,Cash,Street-hail
4,1.0,N,1.0,1.0,1.0,"Creative Mobile Technologies, LLC;",not a store and forward trip,Standard rate,Credit card,Street-hail
...,...,...,...,...,...,...,...,...,...,...
1734046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1734047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1734048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1734049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
final_df.to_csv("final_df.csv",index=False)

In [16]:
from google.colab import files
files.download('final_df.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>